
# BG-NBD and Gamma-Gamma CLTV Prediction


## Business Problem

FLO aims to establish a roadmap for its sales and marketing activities. 
To make medium and long-term plans, the company needs to estimate the potential value existing customers will provide in the future.



## Dataset Description


The dataset contains information derived from past purchasing behavior of OmniChannel (both online and offline) customers who made their last purchases in 2020-2021.

- master_id: Unique customer ID
- order_channel: The channel used for the purchase (Android, iOS, Desktop, Mobile, Offline)
- last_order_channel: The channel used for the most recent purchase
- first_order_date: The date of the first purchase by the customer
- last_order_date: The date of the last purchase by the customer
- last_order_date_online: The date of the last online purchase by the customer
- last_order_date_offline: The date of the last offline purchase by the customer
- order_num_total_ever_online: Total number of online purchases made by the customer
- order_num_total_ever_offline: Total number of offline purchases made by the customer
- customer_value_total_ever_offline: Total amount spent by the customer in offline purchases
- customer_value_total_ever_online: Total amount spent by the customer in online purchases
- interested_in_categories_12: List of categories purchased in the last 12 months


# Tasks

## Task 1: Data Preparation
1. Load the dataset `flo_data_20K.csv` and create a copy of the DataFrame.
2. Define the functions `outlier_thresholds` and `replace_with_thresholds` to suppress outliers.
   Note: Since the CLTV calculation requires frequency values to be integers, round up the lower and upper thresholds using `round()`.
3. Handle any outliers in the columns "order_num_total_ever_online," "order_num_total_ever_offline," "customer_value_total_ever_offline," and "customer_value_total_ever_online."
4. Create new variables representing the total number of purchases and spending for each customer, considering their OmniChannel shopping behavior.
5. Inspect data types and convert any date-related variables to the date format.

## Task 2: Create CLTV Data Structure
1. Use a date two days after the most recent purchase date as the analysis date.
2. Create a new DataFrame `cltv` that contains `customer_id`, `recency_cltv_weekly`, `T_weekly`, `frequency`, and `monetary_cltv_avg`.
   Ensure that the monetary value is expressed as the average value per purchase and that recency and tenure are expressed in weeks.

## Task 3: Build BG/NBD and Gamma-Gamma Models, Calculate CLTV
1. Fit the BG/NBD model.
   a. Predict expected purchases over 3 months and add to the `cltv` DataFrame as `exp_sales_3_month`.
   b. Predict expected purchases over 6 months and add to the `cltv` DataFrame as `exp_sales_6_month`.
2. Fit the Gamma-Gamma model. Estimate the average expected value customers will leave and add to the `cltv` DataFrame as `exp_average_value`.
3. Calculate the 6-month CLTV and add it to the DataFrame as `cltv`.
   a. Standardize the calculated CLTV values and create a new variable `scaled_cltv`.
   b. Observe the top 20 customers with the highest CLTV values.

## Task 4: Create Segments Based on CLTV
1. Segment all customers into 4 groups based on the standardized 6-month CLTV and add the group names to the DataFrame as `cltv_segment`.
2. Provide 6-month action recommendations for 2 of the groups to management.

## Task 5: Functionalize the Entire Process


In [1]:
import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None

## Task 1: Data Preparation

In [4]:
# 1. OmniChannel.csv verisini okuyunuz.Dataframe’in kopyasını oluşturunuz.
df_ = pd.read_excel('eCommerce_data_20k.xlsx')
df = df_.copy()

In [5]:
# 2. Aykırı değerleri baskılamak için gerekli olan outlier_thresholds ve replace_with_thresholds fonksiyonlarını tanımlayınız.
# Not: cltv hesaplanırken frequency değerleri integer olması gerekmektedir.Bu nedenle alt ve üst limitlerini round() ile yuvarlayınız.
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit,0)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit,0)

In [9]:
# 3. "order_num_total_ever_online","order_num_total_ever_offline","customer_value_total_ever_offline","customer_value_total_ever_online" değişkenlerinin
#aykırı değerleri varsa baskılayanız.
# Example conversion if non-numeric types are found:
for col in columns:
    # Attempt to convert to numeric, coerce errors to NaN (which quantile can handle)
    df[col] = pd.to_numeric(df[col], errors='coerce')

columns = ["order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline","customer_value_total_ever_online"]
for col in columns:
    replace_with_thresholds(df, col)

In [10]:

# 4. Omnichannel müşterilerin hem online'dan hemde offline platformlardan alışveriş yaptığını ifade etmektedir.
# Herbir müşterinin toplam alışveriş sayısı ve harcaması için yeni değişkenler oluşturun.
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [11]:
# 5. Değişken tiplerini inceleyiniz. Tarih ifade eden değişkenlerin tipini date'e çeviriniz.
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)

## Task 2: Create CLTV Data Structure

In [13]:
# 1.Veri setindeki en son alışverişin yapıldığı tarihten 2 gün sonrasını analiz tarihi olarak alınız.
df["last_order_date"].max() # 2021-05-30
analysis_date = dt.datetime(2021,6,1)

In [14]:
# 2.customer_id, recency_cltv_weekly, T_weekly, frequency ve monetary_cltv_avg değerlerinin yer aldığı yeni bir cltv dataframe'i oluşturunuz.
cltv_df = pd.DataFrame()
cltv_df["customer_id"] = df["master_id"]
cltv_df["recency_cltv_weekly"] = ((df["last_order_date"]- df["first_order_date"]).astype('timedelta64[D]')) / 7
cltv_df["T_weekly"] = ((analysis_date - df["first_order_date"]).astype('timedelta64[D]'))/7
cltv_df["frequency"] = df["order_num_total"]
cltv_df["monetary_cltv_avg"] = df["customer_value_total"] / df["order_num_total"]

cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5.00,187.87
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21.00,95.88
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5.00,117.06
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2.00,60.98
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2.00,104.99


## Task 3: Build BG/NBD and Gamma-Gamma Models, Calculate CLTV

In [15]:
# 1. BG/NBD modelini kurunuz.
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['frequency'],
        cltv_df['recency_cltv_weekly'],
        cltv_df['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

In [16]:
2.
#a) 3 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_3_month olarak cltv dataframe'ine ekleyiniz.
cltv_df["exp_sales_3_month"] = bgf.predict(4*3,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'])

#b) 6 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_6_month olarak cltv dataframe'ine ekleyiniz.
cltv_df["exp_sales_6_month"] = bgf.predict(4*6,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'])

In [17]:
# 3. ve 6.aydaki en çok satın alım gerçekleştirecek 10 kişiyi inceleyeniz. Fark var mı?
cltv_df.sort_values("exp_sales_3_month",ascending=False)[:10]

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.71,67.29,52.00,153.26,4.66,9.31
15611,4a7e875e-e6ce-11ea-8f44-000d3a38a36f,39.71,40.00,29.00,165.30,3.37,6.75
8328,1902bf80-0035-11eb-8341-000d3a38a36f,28.86,33.29,25.00,NaN,3.14,6.28
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.57,58.71,31.00,228.53,3.08,6.17
14373,f00ad516-c4f4-11ea-98f7-000d3a38a36f,38.00,46.43,27.00,NaN,3.00,6.00
10489,7af5cd16-b100-11e9-9757-000d3a38a36f,103.14,111.86,43.00,157.11,2.98,5.96
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,133.14,147.14,49.00,148.09,2.83,5.66
6756,27310582-6362-11ea-a6dc-000d3a38a36f,62.71,64.14,29.00,168.88,2.79,5.59
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.71,13.00,17.00,259.87,2.78,5.56
10536,e143b6fa-d6f8-11e9-93bc-000d3a38a36f,104.57,113.43,40.00,NaN,2.76,5.53


In [25]:
cltv_df.sort_values("exp_sales_6_month",ascending=False)[:10]

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,cltv,cltv_segment
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.71,67.29,52.00,153.26,4.66,9.31,1519.21,A
15611,4a7e875e-e6ce-11ea-8f44-000d3a38a36f,39.71,40.00,29.00,165.30,3.37,6.75,1201.10,A
8328,1902bf80-0035-11eb-8341-000d3a38a36f,28.86,33.29,25.00,NaN,3.14,6.28,NaN,NaN
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.57,58.71,31.00,228.53,3.08,6.17,1515.10,A
14373,f00ad516-c4f4-11ea-98f7-000d3a38a36f,38.00,46.43,27.00,NaN,3.00,6.00,NaN,NaN
10489,7af5cd16-b100-11e9-9757-000d3a38a36f,103.14,111.86,43.00,157.11,2.98,5.96,999.12,A
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,133.14,147.14,49.00,148.09,2.83,5.66,892.99,A
6756,27310582-6362-11ea-a6dc-000d3a38a36f,62.71,64.14,29.00,168.88,2.79,5.59,1015.99,A
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.71,13.00,17.00,259.87,2.78,5.56,1585.46,A
10536,e143b6fa-d6f8-11e9-93bc-000d3a38a36f,104.57,113.43,40.00,NaN,2.76,5.53,NaN,NaN


In [20]:
# Filter data to meet model assumptions
filtered_df = cltv_df[(cltv_df['frequency'] > 0) & (cltv_df['monetary_cltv_avg'] > 0)]

# Try fitting the model again
ggf = GammaGammaFitter(penalizer_coef=0.1)
ggf.fit(filtered_df['frequency'], filtered_df['monetary_cltv_avg'])
filtered_df["exp_average_value"] = ggf.conditional_expected_average_profit(filtered_df['frequency'],
                                                                           filtered_df['monetary_cltv_avg'])

In [21]:
# 3. 6 aylık CLTV hesaplayınız ve cltv ismiyle dataframe'e ekleyiniz.
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['frequency'],
                                   cltv_df['recency_cltv_weekly'],
                                   cltv_df['T_weekly'],
                                   cltv_df['monetary_cltv_avg'],
                                   time=6,
                                   freq="W",
                                   discount_rate=0.01)
cltv_df["cltv"] = cltv


In [22]:
# CLTV değeri en yüksek 20 kişiyi gözlemleyiniz.
cltv_df.sort_values("cltv",ascending=False)[:20]

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,cltv
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.14,13.14,11.00,696.81,1.97,3.94,3088.11
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.29,9.43,2.00,862.69,0.79,1.59,2277.81
1853,f02473b0-43c3-11eb-806e-000d3a38a36f,17.29,23.14,2.00,835.88,0.68,1.37,1902.29
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.43,34.43,8.00,601.23,1.27,2.53,1759.00
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.71,101.00,7.00,1029.43,0.72,1.44,1744.10
11179,d2e74a36-3228-11eb-860c-000d3a38a36f,1.14,26.29,3.00,750.57,0.78,1.56,1630.46
15516,9083981a-f59e-11e9-841e-000d3a38a36f,63.57,83.86,4.00,1090.36,0.57,1.15,1612.33
9738,3a27b334-dff4-11ea-acaa-000d3a38a36f,40.00,41.14,3.00,837.06,0.68,1.36,1587.98
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.71,13.00,17.00,259.87,2.78,5.56,1585.46
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.29,74.57,16.00,459.75,1.57,3.13,1583.31


## Task 4: Create Segments Based on CLTV

In [23]:
# 1. 6 aylık standartlaştırılmış CLTV'ye göre tüm müşterilerinizi 4 gruba (segmente) ayırınız ve grup isimlerini veri setine ekleyiniz.
# cltv_segment ismi ile atayınız.
cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5.00,187.87,0.97,1.95,450.86,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21.00,95.88,0.98,1.97,205.12,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5.00,117.06,0.67,1.34,193.56,C
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2.00,60.98,0.70,1.40,143.15,C
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2.00,104.99,0.40,0.79,138.87,C


## Task 5 : Functionalization the entire process

In [26]:
def create_cltv_df(dataframe):

    # Veriyi Hazırlama
    columns = ["order_num_total_ever_online", "order_num_total_ever_offline", "customer_value_total_ever_offline","customer_value_total_ever_online"]
    for col in columns:
        replace_with_thresholds(dataframe, col)

    dataframe["order_num_total"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["customer_value_total"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    dataframe = dataframe[~(dataframe["customer_value_total"] == 0) | (dataframe["order_num_total"] == 0)]
    date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)

    # CLTV veri yapısının oluşturulması
    dataframe["last_order_date"].max()  # 2021-05-30
    analysis_date = dt.datetime(2021, 6, 1)
    cltv_df = pd.DataFrame()
    cltv_df["customer_id"] = dataframe["master_id"]
    cltv_df["recency_cltv_weekly"] = ((dataframe["last_order_date"] - dataframe["first_order_date"]).astype('timedelta64[D]')) / 7
    cltv_df["T_weekly"] = ((analysis_date - dataframe["first_order_date"]).astype('timedelta64[D]')) / 7
    cltv_df["frequency"] = dataframe["order_num_total"]
    cltv_df["monetary_cltv_avg"] = dataframe["customer_value_total"] / dataframe["order_num_total"]
    cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

    # BG-NBD Modelinin Kurulması
    bgf = BetaGeoFitter(penalizer_coef=0.001)
    bgf.fit(cltv_df['frequency'],
            cltv_df['recency_cltv_weekly'],
            cltv_df['T_weekly'])
    cltv_df["exp_sales_3_month"] = bgf.predict(4 * 3,
                                               cltv_df['frequency'],
                                               cltv_df['recency_cltv_weekly'],
                                               cltv_df['T_weekly'])
    cltv_df["exp_sales_6_month"] = bgf.predict(4 * 6,
                                               cltv_df['frequency'],
                                               cltv_df['recency_cltv_weekly'],
                                               cltv_df['T_weekly'])

    # # Gamma-Gamma Modelinin Kurulması
    # Filter data to meet model assumptions
    filtered_df = cltv_df[(cltv_df['frequency'] > 0) & (cltv_df['monetary_cltv_avg'] > 0)]

# Try fitting the model again
    ggf = GammaGammaFitter(penalizer_coef=0.1)
    ggf.fit(filtered_df['frequency'], filtered_df['monetary_cltv_avg'])
    filtered_df["exp_average_value"] = ggf.conditional_expected_average_profit(filtered_df['frequency'],
                                                                           filtered_df['monetary_cltv_avg'])

    # Cltv tahmini
    cltv = ggf.customer_lifetime_value(bgf,
                                       cltv_df['frequency'],
                                       cltv_df['recency_cltv_weekly'],
                                       cltv_df['T_weekly'],
                                       cltv_df['monetary_cltv_avg'],
                                       time=6,
                                       freq="W",
                                       discount_rate=0.01)
    cltv_df["cltv"] = cltv

    # CLTV segmentleme
    cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])

    return cltv_df

cltv_df = create_cltv_df(df)


cltv_df.head(10)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5.00,187.87,0.97,1.95,450.86,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21.00,95.88,0.98,1.97,205.12,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5.00,117.06,0.67,1.34,193.56,C
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2.00,60.98,0.70,1.40,143.15,C
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2.00,104.99,0.40,0.79,138.87,C
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,120.86,132.29,3.00,66.95,0.38,0.77,71.81,D
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,32.57,64.86,4.00,93.98,0.65,1.30,158.08,C
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,12.71,54.57,2.00,81.81,0.52,1.04,142.22,C
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,58.43,70.71,5.00,210.94,0.71,1.42,367.82,A
9,1143f032-440d-11ea-8b43-000d3a38a36f,61.71,96.00,2.00,82.98,0.39,0.79,109.53,D
